In [4]:
import sys
sys.path.append('../..')
from src.models import *

%run -i 'train_MLP.py'

Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 2s 146us/sample - loss: 34.4397 - accuracy: 0.2226 - val_loss: 33.9608 - val_accuracy: 1.0000
Epoch 2/50
16000/16000 [==============================] - 2s 109us/sample - loss: 33.6287 - accuracy: 0.1456 - val_loss: 33.1090 - val_accuracy: 0.0000e+00
Epoch 3/50
16000/16000 [==============================] - 2s 104us/sample - loss: 33.1844 - accuracy: 0.1517 - val_loss: 33.0627 - val_accuracy: 0.0000e+00
Epoch 4/50
16000/16000 [==============================] - 2s 103us/sample - loss: 33.0289 - accuracy: 0.4754 - val_loss: 33.0603 - val_accuracy: 0.9923
Epoch 5/50
16000/16000 [==============================] - 2s 106us/sample - loss: 33.0557 - accuracy: 0.6316 - val_loss: 33.0603 - val_accuracy: 0.9620
Epoch 6/50
16000/16000 [==============================] - 2s 103us/sample - loss: 33.0442 - accuracy: 0.7228 - val_loss: 33.0603 - val_accuracy: 0.9872
Epoch 7/50
16000/16000 [=======

KeyboardInterrupt: 

In [2]:
import sys
sys.path.append('../..')
from src.models import *

%run -i 'train_GMF.py'

Train on 16000 samples, validate on 4000 samples
Epoch 1/50
16000/16000 [==============================] - 18s 1ms/sample - loss: 32.9409 - val_loss: 33.0603
Epoch 2/50
 4100/16000 [======>.......................] - ETA: 12s - loss: 34.3078

KeyboardInterrupt: 